In [1]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
import pandas as pd
import copy
import matplotlib.pyplot as plt
import time 

# k-fold cross validation


In this notebook you will use K-fold cross validation to compare two regression models. The two models are Kernel Ridge Regression and Regression Decision Trees. You don't need to understand how these two models work for now and can just use the provided fit_and_predict functions to get predictions for each of the models. 

The aim of this notebook is to practice K-fold cross validation. It is split into 5 parts:
1. Create a function to split the data into k folds 
2. Performance Metric - create a function to calculate RMSE
3. Create a function to run cross validation on the k-folds
4. Run cross validation for k={2, ..., 100}, timing each one 
5. Answer the questions in a markdown cell 

**NOTE:** Make sure you restart the kernel and re-run the notebook just before you submit it so your answers are consistant. All the packages you need are already imported into the notebook. 



## Data

We will use a sklearn toy dataset. The input values for this dataset are ten baseline variables measured in diabetes patients and the output is a quantitative measure of disease progression one year later. You can read more about the data [here](https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset).

In [2]:
# Do not edit this cell 

X_, y_ = load_diabetes(return_X_y=True)

# standardise the data to help us fit the data (this will be covered later in the course)

scaler = preprocessing.StandardScaler().fit(X_)
X = scaler.transform(X_)[:300, :]

scaler_y = preprocessing.StandardScaler().fit(y_.reshape(-1, 1))
y = scaler_y.transform(y_.reshape(-1, 1))[:300, :]
print(y.shape)

# to ensure the data stays in the correct order, we will work with dataframes 

columns = [f'x_{i}' for i in range(X.shape[1])] + ['y']
x_columns = [f'x_{i}' for i in range(X.shape[1])]
data = pd.DataFrame(data= np.concatenate([X, y.reshape(-1, 1)], axis=1), columns=columns)
print(data.head())

(300, 1)
        x_0       x_1       x_2       x_3       x_4       x_5       x_6  \
0  0.800500  1.065488  1.297088  0.459841 -0.929746 -0.732065 -0.912451   
1 -0.039567 -0.938537 -1.082180 -0.553505 -0.177624 -0.402886  1.564414   
2  1.793307  1.065488  0.934533 -0.119214 -0.958674 -0.718897 -0.680245   
3 -1.872441 -0.938537 -0.243771 -0.770650  0.256292  0.525397 -0.757647   
4  0.113172 -0.938537 -0.764944  0.459841  0.082726  0.327890  0.171178   

        x_7       x_8       x_9         y  
0 -0.054499  0.418531 -0.370989 -0.014719  
1 -0.830301 -1.436589 -1.938479 -1.001659  
2 -0.054499  0.060156 -0.545154 -0.144580  
3  0.721302  0.476983 -0.196823  0.699513  
4 -0.054499 -0.672502 -0.980568 -0.222496  


In [3]:
# Do not edit 

def fit_and_predict_KRR(train, validate):
    """fit a Kernel Ridge Regression Model on the training data and predict the y values of the validation X data.
    :param train: pandas dataframe containing the training data
    :param validate: pandas dataframe containing the validation data 
    :return: predictions at the validation X points Mx1 numpy array"""
    X_train = train[x_columns].to_numpy()
    y_train = train['y'].to_numpy()
    X_val = validate[x_columns].to_numpy()
        
    KRR = KernelRidge(alpha=0.1, kernel='rbf', gamma=0.2,  degree=100)
    KRR.fit(X_train, y_train)
    return KRR.predict(X_val)
    
    
def fit_and_predict_DT(train, validate):
    """fit a Regression Decision Tree on the training data and predict the y values of the validation X data.
    :param train: pandas dataframe containing the training data
    :param validate: pandas dataframe containing the validation data 
    :return: predictions at the validation X points Mx1 numpy array"""
    
    X_train = train[x_columns].to_numpy()
    y_train = train['y'].to_numpy()
    X_val = validate[x_columns].to_numpy()
        
    DTree = DecisionTreeRegressor(max_depth=6)
    DTree.fit(X_train, y_train)
    return DTree.predict(X_val)


## Part 1 : Creating the folds

### TO DO:

Complete the gaps to crete a function that split the data into k folds. 

**MARKS: 2 Marks for correctly creating list of how long each fold should be**

In [4]:

def k_folds(data, k):
    """function that returns a list of k folds of the data"""
    
    ############################
    # Create list of how long each fold should be. The folds should be as even as possible in number, but some may
    # need to have an extra data point if the total number of data points isn't divisible by n
    len_folds = [int(sum(x)) for x in np.array_split(np.ones(len(data)), k)]
    ############################

    folds = []
    for i in range(k):
        data_ss = data.sample(n=len_folds[i], random_state=20)
        data = data.drop(data_ss.index)
        folds.append(data_ss)

    return folds 

## Part 2:  Performance Metric

### TO DO:
Write a function that calculates the root mean squared error between predictions and the true y values. Both inputs should be numpy arrays and the function should return a float. 

**MARKS: 1 mark for correctly writing function**

In [5]:
def rmse(prediction, true):
    return np.sqrt(np.mean(np.square(prediction-true)))

## Part 3: Cross Valiation

### TO DO:

Create a function to run the cross validation on both the models by filling out the gaps in the function below. This function will return the average RMSE for each of the models. 

**MARKS: 6 Marks, one for each of the code blocks to be completed in the function**


In [6]:
def cross_validation(folds):
    folds = copy.copy(folds) # this creates a new variable which is a copy of folds

    rmses_KRR = []  # list to collect the rmses for each fold for the Kernel Ridge Regression
    rmses_DT = []   # list to collect the rmses for each fold for the Decision Tree
    for i, fold in enumerate(folds):
        
        ############################
        # Write code to create the training and validation sets as DataFrames
        
        train = pd.concat(folds[:i]+folds[(i+1):])
#         train = pd.concat([folds.pop(i)])
        validate = fold
        
        ############################
        
        ############################
        # Use the fit_and_predict functions to create new columns in the validation set for the predictions 
        # for each model with headings ['KRR_predictions', 'DT_predictions'].
        
        validate['KRR_predictions'] = fit_and_predict_KRR(train, fold)
        validate['DT_predictions'] = fit_and_predict_DT(train, fold)
        
        ############################
        
        ############################
        # calculate the rmse for the two models and append to rmses_KRR and rmses_DT
        
        rmses_KRR.append(rmse(validate['KRR_predictions'].to_numpy(), validate['y'].to_numpy()))
        rmses_DT.append(rmse(validate['DT_predictions'].to_numpy(), validate['y'].to_numpy()))
        
        ############################

    RMSE_KRR = np.mean(rmses_KRR) # calculate the average RMSEs for kernel ridge regression
    RMSE_DT = np.mean(rmses_DT)# calculate the average RMSEs for the decision tree 
    return RMSE_KRR, RMSE_DT


### TO DO: 
For k = 100 calculate the RMSE for each model and print the solutions 

**MARKS: 2 Marks, one for each of the RMSEs. RMSE_KRR = 0.08646068038691881 and RMSE_DT = 0.0. Note: I think all answers should be the same but it's possible they may vary slightly depending on the learner's code**

In [7]:
folds =  k_folds(data, 100)
RMSE_KRR, RMSE_DT = cross_validation(copy.copy(folds))
print(RMSE_KRR, RMSE_DT)

0.7953413904964531 0.7691086835088461


## Part 4: Cross Validation for Different Values of k

### TO DO:

for k in {2, ..., 100} divide the data into k folds and then run cross validation. Save the results for each run in two lists (one for each model) and then plot a graph of k on the x-axis and RMSE on the y_axis. 

Use the time function (example in cell below) to time how long the cross validation takes for each value of k. Make a plot of the time against the value of k. 

**MARKS: 6 Marks. 2 Marks for the correct iteration. 2 Marks for the correct plot of RMSEs, 2 Marks for correct plot of time**

In [24]:
# time function example

start = time.time()
print('hello')
end = time.time()
print(end - start)

hello
0.0005736351013183594


In [8]:
K=100
KRRs = []
DTs = []
times = []
for k in range(2, K):
    start = time.time()
    folds = k_folds(data, k)
    RMSE_KRR, RMSE_DT = cross_validation(folds)
    end = time.time()
    KRRs.append(RMSE_KRR)
    DTs.append(RMSE_DT)
    times.append(end - start)
    
    
plt.plot(list(range(2,K)), KRRs, label='KRR')
plt.plot(list(range(2,K)), DTs, label='DT')
plt.legend()
plt.title('RMSEs')
plt.ylabel('RMSE')
plt.xlabel('k')

fig = plt.figure()
plt.plot(list(range(2,K)), times)
plt.title('time to compute')
plt.ylabel('time (s)')
plt.xlabel('k')

KeyboardInterrupt: 

In [9]:
print(KRRs[:10], DTs[:10])


[0.9298036669683769, 0.902648483368005, 0.8652785942241344, 0.880885028586549, 0.8899224962701903, 0.8869068524213862, 0.8758217838151858, 0.8598491907287679, 0.8830630945129861, 0.8750251574652751] [0.9787478515561953, 0.9916146510792903, 0.9140320599536105, 0.8683230577899537, 0.972782871555304, 0.9154003128920563, 0.8580610886435834, 0.9059931739819326, 0.8958854699985583, 0.888670483741923]


## Part 5 : Questions

### TO DO:
Answer the following questions in a markdown cell

1. Which model would you select based on your cross validation results? Why? 
2. Looking at the two plots you made, what are the benefits and drawbacks of increasing k? 


### ANSWER:

1. The students should select the Decision Tree Regression model as the RMSE is lower 
2. As k increases, we get more consistant results which is good for reproducibility. However, this is at the cost of computational time. 

**MARKS: 3 Marks, 1 mark for fist question, 2 marks for recognising that results get more consistant but comp time increases as k increases**